In [1]:
import os, json
import pandas as pd
import timeit
import sqlalchemy as alch
import numpy as np
from datetime import datetime
#import matplotlib.pyplot as plt
#import matplotlib.dates as mdates
from collections import Counter
import csv

# VARIABLES

In [69]:
ntopics = 20
model_db = 'sqlite:///../dbs/jobs_LDA_20_clustered.db'
# use 1 for LDA table, 1 for PCA 2 for t-SNE in a D2V table
col_index = 1

# PART 1 - READING FROM THE DATABASES

In [70]:
# open the databse with nutslocations for the jobs
engine = alch.create_engine('sqlite:///jobs_nutscodes_names_v2.db', echo=False)

In [71]:
%%time
with engine.connect() as connection:
    result = connection.execute("select count(*) from jobs")
    number_of_rows = result.first()[0]
print(number_of_rows)

3114612
Wall time: 79.8 ms


In [72]:
%%time
jobs = [None] * number_of_rows

with engine.connect() as connection:
    result = connection.execute("select jobid, nutsCode from jobs")
    i = 0
    nones = 0
    for row in result:
        jobs[i] = [row[0], row[1]]
        i+=1  
        if i % 100000 == 0:
            print(i,"rows processed")

100000 rows processed
200000 rows processed
300000 rows processed
400000 rows processed
500000 rows processed
600000 rows processed
700000 rows processed
800000 rows processed
900000 rows processed
1000000 rows processed
1100000 rows processed
1200000 rows processed
1300000 rows processed
1400000 rows processed
1500000 rows processed
1600000 rows processed
1700000 rows processed
1800000 rows processed
1900000 rows processed
2000000 rows processed
2100000 rows processed
2200000 rows processed
2300000 rows processed
2400000 rows processed
2500000 rows processed
2600000 rows processed
2700000 rows processed
2800000 rows processed
2900000 rows processed
3000000 rows processed
3100000 rows processed
Wall time: 8.2 s


In [73]:
# open the database with topics for the jobs
engine2 = alch.create_engine(model_db, echo=False)

In [74]:
%%time
with engine2.connect() as connection:
    result = connection.execute("SELECT COUNT(*) FROM jobs")
    number_of_rows = result.first()[0]
print(number_of_rows)

3279172
Wall time: 93.4 ms


In [75]:
%%time
topics = [None] * number_of_rows

with engine2.connect() as connection:
    result = connection.execute("select * from jobs")
    i = 0
    nones = 0
    for row in result:
        topics[i] = [row[0], row[col_index]]
        i+=1  
        if i % 100000 == 0:
            print(i,"rows processed")

100000 rows processed
200000 rows processed
300000 rows processed
400000 rows processed
500000 rows processed
600000 rows processed
700000 rows processed
800000 rows processed
900000 rows processed
1000000 rows processed
1100000 rows processed
1200000 rows processed
1300000 rows processed
1400000 rows processed
1500000 rows processed
1600000 rows processed
1700000 rows processed
1800000 rows processed
1900000 rows processed
2000000 rows processed
2100000 rows processed
2200000 rows processed
2300000 rows processed
2400000 rows processed
2500000 rows processed
2600000 rows processed
2700000 rows processed
2800000 rows processed
2900000 rows processed
3000000 rows processed
3100000 rows processed
3200000 rows processed
Wall time: 8.79 s


In [76]:
%%time
# filter out the entries in topics not matched by an entry in jobs 
jobs_located = [item[0] for item in jobs]
jobs_with_topics = [item[0] for item in topics]

tmp = set(jobs_with_topics) 
jobs_considered = [value for value in jobs_located if value in tmp]


Wall time: 901 ms


In [77]:
jobs_with_topics[35]
jobs_located[36]

37000037

In [78]:
# Open the NOMIS dataset for additional stats and build a dictionary
dem_info = {}

with open('NOMIS-dataset.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        try:
            if 'UK' in row[1]:
                # if ukm2, switch it to 7
                if row[1][:4] == 'UKM2':
                    row[1] = 'UKM7'+row[1][4:]
                elif row[1][:4] == 'UKM3':
                    row[1] = 'UKM8'+row[1][4:]
                    entry = {'population':row[3],'employmentrate':row[4],'employeerate':row[8],'unemploymentrate':row[12]}
                    dem_info[row[1]] = entry
                    row[1] = 'UKM9'+row[1][4:]
                entry = {'population':row[3],'employmentrate':row[4],'employeerate':row[8],'unemploymentrate':row[12]}
                dem_info[row[1]] = entry
        except:
            pass

In [79]:
print(dem_info)

{'UKC': {'population': '1635500', 'employmentrate': '71.1', 'employeerate': '62.6', 'unemploymentrate': '5.8'}, 'UKD': {'population': '4497100', 'employmentrate': '74.9', 'employeerate': '64.9', 'unemploymentrate': '4.1'}, 'UKE': {'population': '3379400', 'employmentrate': '74.0', 'employeerate': '64.4', 'unemploymentrate': '4.2'}, 'UKF': {'population': '2947000', 'employmentrate': '76.8', 'employeerate': '66.6', 'unemploymentrate': '3.7'}, 'UKG': {'population': '3613600', 'employmentrate': '73.9', 'employeerate': '63.8', 'unemploymentrate': '5.0'}, 'UKH': {'population': '3776800', 'employmentrate': '77.9', 'employeerate': '66.5', 'unemploymentrate': '3.3'}, 'UKI': {'population': '6082300', 'employmentrate': '75.1', 'employeerate': '61.5', 'unemploymentrate': '4.8'}, 'UKJ': {'population': '5577000', 'employmentrate': '79.6', 'employeerate': '67.3', 'unemploymentrate': '3.2'}, 'UKK': {'population': '3336000', 'employmentrate': '79.2', 'employeerate': '66.3', 'unemploymentrate': '3.3'}, 

In [80]:
%%time
# build a list containing all jobs considered including topic and location
newJobList = [None]*len(jobs_considered)
# counters
c1 = 0
c2 = 0
for i in range(len(jobs_considered)):
   
    while(jobs_considered[i] != jobs_located[c1]):
        c1+=1

    while(jobs_considered[i] != jobs_with_topics[c2]):
        c2+=1

    newJobList[i] = [jobs_considered[i], jobs[c1][1], topics[c2][1]]

Wall time: 4.44 s


In [81]:
print(newJobList[:10])

[[37000000, 'UKH12', 6], [37000001, 'UKJ25', 11], [37000002, 'UKE32', 12], [37000003, 'UKI34', 6], [37000004, 'UKD34', 15], [37000005, 'UKJ25', 16], [37000006, 'UKI34', 4], [37000007, 'UKG33', 3], [37000008, 'UKK12', 1], [37000009, 'UKI34', 17]]


# PART 2 - BUILDING STATS

In [82]:
%%time
nutscodes = []
with open('NUTS_Level_3__January_2018__Boundaries.GEOJSON', 'r') as f:
        geo_data = f.read()
        geo_data = json.loads(geo_data)

for f in geo_data['features']:    
        nutscodes.append(f['properties']['nuts318cd'])

Wall time: 4.99 s


In [83]:
#get all unique codes - there should be 180
nuts3 = set(nutscodes)
nuts2 = set({x[:4] for x in nuts3})
nuts1 = set({x[:3] for x in nuts2})
print(len(nuts3))
print(len(nuts2))
print(len(nuts1))


179
42
12


In [84]:
print(nuts3)

{'UKJ37', 'UKG23', 'UKM61', 'UKM78', 'UKK11', 'UKM77', 'UKH32', 'UKD72', 'UKJ46', 'UKG22', 'UKM65', 'UKM95', 'UKG39', 'UKN09', 'UKF15', 'UKD42', 'UKI31', 'UKC12', 'UKM71', 'UKN15', 'UKI63', 'UKM62', 'UKH31', 'UKK30', 'UKF22', 'UKM84', 'UKD47', 'UKE22', 'UKD33', 'UKE42', 'UKE44', 'UKF30', 'UKN14', 'UKL23', 'UKE41', 'UKG33', 'UKL15', 'UKI32', 'UKH11', 'UKG38', 'UKD45', 'UKE31', 'UKJ36', 'UKD34', 'UKD11', 'UKE21', 'UKI44', 'UKC13', 'UKK15', 'UKH14', 'UKJ21', 'UKE12', 'UKH34', 'UKD61', 'UKK42', 'UKD71', 'UKG36', 'UKI74', 'UKM76', 'UKD37', 'UKD12', 'UKD73', 'UKF25', 'UKH15', 'UKH35', 'UKI45', 'UKI62', 'UKJ14', 'UKJ32', 'UKJ28', 'UKI53', 'UKC11', 'UKG31', 'UKK23', 'UKJ12', 'UKM93', 'UKC14', 'UKJ41', 'UKD63', 'UKN07', 'UKH16', 'UKL16', 'UKD41', 'UKF16', 'UKM63', 'UKM66', 'UKJ25', 'UKK43', 'UKK12', 'UKG12', 'UKC23', 'UKL11', 'UKK13', 'UKL18', 'UKF24', 'UKL12', 'UKN16', 'UKJ34', 'UKK41', 'UKH17', 'UKL14', 'UKK21', 'UKJ13', 'UKH24', 'UKE45', 'UKI61', 'UKI75', 'UKD35', 'UKC21', 'UKL24', 'UKF12', 

In [85]:
%%time
# count jobs in each zone
counts_nuts3 = {}

for area in nuts3:
    #counters
    c=0
    topic_counts = [0]*ntopics
    for element in newJobList:
        if element[1] == area:
            c+=1
            topic_counts[element[2]]+=1
    try:
        counts_nuts3[area] = {'count':c, 'topic_spread':topic_counts,'dem_stats':dem_info[area]}
    except:
        print(area)
        counts_nuts3[area] = {'count':c, 'topic_spread':topic_counts,'dem_stats':'NA'}


UKN09
UKN15
UKN14
UKN07
UKN16
UKN08
UKN06
UKN13
UKN12
UKN11
UKN10
Wall time: 44.8 s


In [86]:
%%time
counts_nuts2 = {}

for area in nuts2:
    #counters
    c=0
    topic_counts = [0]*ntopics
    for element in newJobList:
        if element[1][:4] == area:
            c+=1
            topic_counts[element[2]]+=1
    try:
        counts_nuts2[area] = {'count':c, 'topic_spread':topic_counts,'dem_stats':dem_info[area]}
    except:
        print(area)
        counts_nuts2[area] = {'count':c, 'topic_spread':topic_counts,'dem_stats':'NA'}

    

UKN1
UKN0
Wall time: 18.8 s


In [87]:
%%time
counts_nuts1 = {}

for area in nuts1:
    #counters
    c=0
    topic_counts = [0]*ntopics
    for element in newJobList:
        if element[1][:3] == area:
            c+=1
            topic_counts[element[2]]+=1
    try:
        counts_nuts1[area] = {'count':c, 'topic_spread':topic_counts,'dem_stats':dem_info[area]}
    except:
        print(area)
        counts_nuts1[area] = {'count':c, 'topic_spread':topic_counts,'dem_stats':'NA'}

    

UKN
Wall time: 5.75 s


In [88]:
print(counts_nuts1)

{'UKC': {'count': 66838, 'topic_spread': [4070, 5271, 1174, 5325, 2610, 3537, 4231, 2309, 1268, 4701, 5131, 4607, 3917, 4431, 2911, 1147, 1241, 3760, 713, 4484], 'dem_stats': {'population': '1635500', 'employmentrate': '71.1', 'employeerate': '62.6', 'unemploymentrate': '5.8'}}, 'UKH': {'count': 353215, 'topic_spread': [28829, 23283, 3482, 42126, 14170, 7284, 25980, 21259, 9550, 15328, 26491, 25021, 19581, 21785, 12667, 8130, 4945, 18403, 5828, 19073], 'dem_stats': {'population': '3776800', 'employmentrate': '77.9', 'employeerate': '66.5', 'unemploymentrate': '3.3'}}, 'UKD': {'count': 321031, 'topic_spread': [22973, 21695, 3166, 29081, 21062, 14948, 14445, 23918, 10046, 11133, 22305, 27069, 16398, 12016, 13156, 11114, 10658, 16494, 4848, 14506], 'dem_stats': {'population': '4497100', 'employmentrate': '74.9', 'employeerate': '64.9', 'unemploymentrate': '4.1'}}, 'UKL': {'count': 67178, 'topic_spread': [5503, 5813, 617, 5135, 3148, 3189, 3746, 2588, 868, 4358, 5769, 3646, 4196, 4848, 313

In [89]:
# write out JS file:
data = {
        'nuts1':counts_nuts1,
        'nuts2':counts_nuts2,
        'nuts3':counts_nuts3
        }

with open('map_data.js', 'w') as outfile:
    outfile.write('map_data = ')
    json.dump(data, outfile)